In [5]:
# Importando a biblioteca necessária
import pandas as pd

# Carregar o dataset com delimitador e codificação corretos
dataset = pd.read_csv(
    'DataSet.dsv', 
    delimiter=';', 
    encoding='latin1', 
    dtype={'CO_PA_AUTORIZ': str, 
           'CO_PA_CNSPAC': str,
           'CO_PA_PROC_ID': str,
           'CO_PA_MVM': str,
           'CO_PA_CNSMED': str},
    low_memory=False
)

# Análise inicial dos dados
# 1. Quantidade total de registros no conjunto de dados
total_registros = len(dataset)
print(f"Quantidade total de registros no conjunto de dados: {total_registros}")

# 2. Quantidade e percentual de cada agrupamento de CO_PA_DOCORIG
agrupamento_docorig = dataset['CO_PA_DOCORIG'].value_counts(dropna=False)
percentual_docorig = (agrupamento_docorig / total_registros) * 100
print("\nQuantidade e percentual de cada agrupamento de CO_PA_DOCORIG:")
print(pd.DataFrame({'Quantidade': agrupamento_docorig, 'Percentual': percentual_docorig}))

# 2. Quantidade e percentual de cada agrupamento de CO_PA_AUTORIZ
agrupamento_autoriz = dataset['CO_PA_AUTORIZ'].value_counts(dropna=False)
percentual_autoriz = (agrupamento_autoriz / total_registros) * 100
print("\nQuantidade e percentual de cada agrupamento de CO_PA_AUTORIZ:")
print(pd.DataFrame({'Quantidade': agrupamento_autoriz, 'Percentual': percentual_autoriz}))

# 2. Quantidade e percentual de cada agrupamento de CO_PA_CNSPAC
agrupamento_cns = dataset['CO_PA_CNSPAC'].value_counts(dropna=False)
percentual_cns = (agrupamento_cns / total_registros) * 100
print("\nQuantidade e percentual de cada agrupamento de CO_PA_CNSPAC:")
print(pd.DataFrame({'Quantidade': agrupamento_cns, 'Percentual': percentual_cns}))



# 3. Quantidade e percentual de registros onde CO_PA_AUTORIZ é igual a '000000000000000' e diferente
autoriz_zeros = dataset[dataset['CO_PA_AUTORIZ'] == '000000000000000'].shape[0]
autoriz_nao_zeros = total_registros - autoriz_zeros
percentual_autoriz_zeros = (autoriz_zeros / total_registros) * 100
percentual_autoriz_nao_zeros = (autoriz_nao_zeros / total_registros) * 100

print("\nQuantidade e percentual de registros em relação ao CO_PA_AUTORIZ:")
print(f"CO_PA_AUTORIZ igual a '000000000000000': {autoriz_zeros} registros ({percentual_autoriz_zeros:.2f}%)")
print(f"CO_PA_AUTORIZ diferente de '000000000000000': {autoriz_nao_zeros} registros ({percentual_autoriz_nao_zeros:.2f}%)")

# 4. Quantidade e percentual de registros onde CO_PA_CNSPAC é nulo e não nulo
cnspac_nulo = dataset['CO_PA_CNSPAC'].isnull().sum()
cnspac_nao_nulo = total_registros - cnspac_nulo
percentual_cnspac_nulo = (cnspac_nulo / total_registros) * 100
percentual_cnspac_nao_nulo = (cnspac_nao_nulo / total_registros) * 100

print("\nQuantidade e percentual de registros em relação ao CO_PA_CNSPAC:")
print(f"CO_PA_CNSPAC nulo: {cnspac_nulo} registros ({percentual_cnspac_nulo:.2f}%)")
print(f"CO_PA_CNSPAC não nulo: {cnspac_nao_nulo} registros ({percentual_cnspac_nao_nulo:.2f}%)")

# **Continuação da análise conforme o código original para pacientes identificados**
# 5. Filtrar para pacientes identificados
pacientes_identificados = dataset[dataset['CO_PA_CNSPAC'] != '000000000000000']

# 6. Filtrar registros pelos códigos de procedimentos
biopsia_prostata = pacientes_identificados[pacientes_identificados['CO_PA_PROC_ID'] == '0201010410']
exame_anatomopatologico = pacientes_identificados[pacientes_identificados['CO_PA_PROC_ID'] == '0203020030']

# 7. Encontrar pacientes que fizeram ambos os procedimentos
pacientes_both = pd.merge(
    biopsia_prostata[['CO_PA_CNSPAC', 'CO_PA_MVM', 'NU_PA_QTDAPR', 'NU_PA_VALAPR']],
    exame_anatomopatologico[['CO_PA_CNSPAC', 'CO_PA_MVM', 'NU_PA_QTDAPR', 'NU_PA_VALAPR']],
    on='CO_PA_CNSPAC',
    suffixes=('_biopsia', '_anatomopatologico')
)

# 8. Função para limpar e converter strings de valores monetários
def limpar_e_converter(valor):
    if isinstance(valor, str):
        valor = valor.replace('.', '').replace(',', '.')
    try:
        return float(valor)
    except ValueError:
        return 0.0  # Retornar 0.0 se a conversão falhar

# 9. Aplicar a função nas colunas de valores monetários
pacientes_both['NU_PA_QTDAPR_biopsia'] = pacientes_both['NU_PA_QTDAPR_biopsia'].apply(limpar_e_converter)
pacientes_both['NU_PA_VALAPR_biopsia'] = pacientes_both['NU_PA_VALAPR_biopsia'].apply(limpar_e_converter)
pacientes_both['NU_PA_QTDAPR_anatomopatologico'] = pacientes_both['NU_PA_QTDAPR_anatomopatologico'].apply(limpar_e_converter)
pacientes_both['NU_PA_VALAPR_anatomopatologico'] = pacientes_both['NU_PA_VALAPR_anatomopatologico'].apply(limpar_e_converter)

# 10. Calcular a soma das quantidades e valores aprovados
pacientes_both['SOMA_NU_PA_QTDAPR'] = pacientes_both['NU_PA_QTDAPR_biopsia'] + pacientes_both['NU_PA_QTDAPR_anatomopatologico']
pacientes_both['SOMA_NU_PA_VALAPR'] = pacientes_both['NU_PA_VALAPR_biopsia'] + pacientes_both['NU_PA_VALAPR_anatomopatologico']

# 11. Identificar inconsistências na ordem dos procedimentos
pacientes_inconsistentes = pacientes_both[pacientes_both['CO_PA_MVM_anatomopatologico'] < pacientes_both['CO_PA_MVM_biopsia']]

# 12. Contagem e exibição de inconsistências
num_pacientes_both = pacientes_both['CO_PA_CNSPAC'].nunique()
num_pacientes_inconsistentes = pacientes_inconsistentes['CO_PA_CNSPAC'].nunique()

# Exibir dados de meses de procedimentos e soma detalhada
meses_procedimentos = pacientes_both.groupby('CO_PA_CNSPAC')[['CO_PA_MVM_biopsia', 'CO_PA_MVM_anatomopatologico']].agg(list)
resultado_final = pacientes_both[['CO_PA_CNSPAC', 'CO_PA_MVM_biopsia', 'SOMA_NU_PA_QTDAPR', 'SOMA_NU_PA_VALAPR',
                                  'NU_PA_QTDAPR_biopsia', 'NU_PA_VALAPR_biopsia', 
                                  'NU_PA_QTDAPR_anatomopatologico', 'NU_PA_VALAPR_anatomopatologico']]

# 13. Avaliar situações regulares e irregulares
situacoes = pacientes_both[['CO_PA_CNSPAC', 'NU_PA_QTDAPR_biopsia', 'NU_PA_QTDAPR_anatomopatologico']].copy()
situacoes['Situacao'] = 'Irregular'  # Define default como Irregular
situacoes.loc[(situacoes['NU_PA_QTDAPR_biopsia'] == 1) & (situacoes['NU_PA_QTDAPR_anatomopatologico'] == 1), 'Situacao'] = 'Regular'

# 14. Contar registros regulares e irregulares
contagem_situacoes = situacoes['Situacao'].value_counts()
percentual_situacoes = (contagem_situacoes / total_registros) * 100

# 15. Preparar DataFrame para exportação
resultado_situacoes = pd.DataFrame({
    'Quantidade': contagem_situacoes,
    'Percentual': percentual_situacoes
})

# 16. Exportar os resultados para Excel
meses_procedimentos.to_excel('meses_procedimentos.xlsx', index=True)
resultado_final.to_excel('resultado_final.xlsx', index=False)
pacientes_inconsistentes.to_excel('pacientes_inconsistentes.xlsx', index=False)
resultado_situacoes.to_excel('resultado_situacoes.xlsx')

print("\nResultados exportados para Excel com sucesso!")

Quantidade total de registros no conjunto de dados: 91444

Quantidade e percentual de cada agrupamento de CO_PA_DOCORIG:
   Quantidade  Percentual
I       90976   99.488211
C         468    0.511789

Quantidade e percentual de cada agrupamento de CO_PA_AUTORIZ:
               Quantidade  Percentual
0000000000000       87792   96.006299
N o informada           4    0.004374
Não Informada           3    0.003281
0323129000869           1    0.001094
0223129001011           1    0.001094
...                   ...         ...
0123129000805           1    0.001094
0123129000599           1    0.001094
0123129000234           1    0.001094
0123129000508           1    0.001094
3523257912086           1    0.001094

[3648 rows x 2 columns]

Quantidade e percentual de cada agrupamento de CO_PA_CNSPAC:
                 Quantidade  Percentual
                       1904    2.082149
NaN                     468    0.511789
898000105155723          49    0.053585
898000077440252          44    0.04